In [2]:
import pandas as pd
import numpy as np

In [128]:
def validate_isin(column,values_to_check,df):
    isin_validation = df[column].isin(values_to_check)
    return isin_validation

def validate_pattern(column,pattern,df):
    pattern_matched = df[column].str.match(pattern)
    return pattern_matched
    
def validate_bounded(column,lower_bound,upper_bound,df):
    bounded_range = (df[column] >= lower_bound) & (df[column] <= upper_bound)
    return bounded_range
    

In [ ]:
# list(df1['Category'].value_counts().keys())

In [132]:
df1= pd.read_csv('GooglePlay.csv')
# df2 = pd.read_csv("Playstore_final.csv")

invalid={}
valid_Type= ['Free', 'Paid']
df=df1[df1['Type' ].notna()]
invalid["Type"] = df[~validate_isin('Type',valid_Type, df)]

valid_Category=['FAMILY', 'GAME', 'TOOLS', 'MEDICAL', 'BUSINESS', 'PRODUCTIVITY', 'PERSONALIZATION', 'COMMUNICATION','SPORTS', 'LIFESTYLE', 'FINANCE', 'HEALTH_AND_FITNESS', 'PHOTOGRAPHY','SOCIAL', 'NEWS_AND_MAGAZINES','SHOPPING', 'TRAVEL_AND_LOCAL','DATING', 'BOOKS_AND_REFERENCE', 'VIDEO_PLAYERS', 'EDUCATION','ENTERTAINMENT', 'MAPS_AND_NAVIGATION', 'FOOD_AND_DRINK', 'HOUSE_AND_HOME','AUTO_AND_VEHICLES','LIBRARIES_AND_DEMO', 'WEATHER', 'ART_AND_DESIGN', 'EVENTS', 'PARENTING','COMICS','BEAUTY']
df=df1[df1['Category' ].notna()]
invalid["Category"]= df[~validate_isin('Category',valid_Category, df)]

df=df1[df1['Installs' ].notna()]
valid_Installs = '\d{1,3}(,\d{3})*\+|0'
invalid['Installs'] = df1[~validate_pattern('Installs',valid_Installs, df)]

pattern_App= '[a-z]*'
invalid['App'] = df1[validate_pattern('App',pattern_App, df)]

df=df1[df1['Last Updated' ].notna()]
pattern_Last_Updated= '\d{1,2}-[(Jan)|(Aug)|Dec|Feb|Jul|Apr|Aug|Mar|May|Nov|Sep|Oct]{3}-\d{2}'
invalid['Last Updated'] = df[~validate_pattern('Last Updated',pattern_Last_Updated, df)]

df=df1[df1['Price' ].notna()]
pattern_Price= '(\$?\d+(\.\d{2})?|0)'
invalid['Price']= df[~validate_pattern('Price',pattern_Price, df)]

df=df1[df1['Genres' ].notna()]
pattern_Genres= '([A-Z][a-z]* ?(&;)?)+' 
invalid['Genres'] = df[~validate_pattern('Genres',pattern_Genres, df)]

valid_Content_Rating= ['Everyone','Teen', 'Mature 17+', 'Everyone 10+', 'Adults only 18+', 'Unrated']
df=df1[df1['Content Rating' ].notna()]
invalid['Content Rating'] = df[~validate_isin('Content Rating',valid_Content_Rating, df)]

df=df1[df1['Rating' ].notna()]
invalid['Rating'] = df[~validate_bounded('Rating',0,5, df)]

df=df1[df1['Reviews' ].notna()]
invalid['Reviews'] = df[~validate_bounded('Reviews',0,np.inf, df)]

df=df1[df1['Size' ].notna()]
pattern_Size='\d+(\.\d+)?[MK]?'
invalid['Size'] = df[~validate_pattern('Size',pattern_Size, df)]

df1['Current Ver'] = df1['Current Ver'].astype(str)

df=df1[df1['Current Ver' ].notna()]
valid_Current_Ver = '^\d+(\.\d+)*'
invalid['Current Ver'] = df[~validate_pattern('Current Ver',valid_Current_Ver, df)]

df1['Android Ver'] = df1['Android Ver'].astype(str)

df=df1[df1['App' ].notna()]

df=df1[df1['Android Ver' ].notna()]
valid_Android_Ver = '\d+(\.\d+)*\s*-\s*\d+(\.\d+)*|\d+(\.\d+)*\s+and\s+up'
invalid['Android Ver' ]= df[~validate_pattern('Android Ver',valid_Android_Ver, df)]




In [ ]:
sum=0
quality_df = pd.DataFrame(index=list(df1.columns),columns=['Attribute', 'Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy', 'Null Values', 'Records'])

for column in df1.columns:
    quality_df['Records'][column] = len(df1)


for column in df1.columns:
    if column != 'Unnamed: 0':  
        quality_df['Null Values'][column]=df1[column].isnull().sum()

for column in df1.columns:
    if column != 'Unnamed: 0':  
        missing=df1[column].isnull().sum()
        sum =(  ((len(df1['App']) - (len(invalid[column])+ missing) )/ len(df1['App'])) * 100)
        quality_df['Accuracy'][column]=sum

# len(quality_df['Null Values'])
# quality_df['Null Values']
# len(df1[])


In [63]:
df2 = pd.read_csv("Playstore_final.csv")

C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_8368\676857778.py:1: DtypeWarning: Columns (7,12,29,30,33,35,37,38,40,42,43,44,46,47,48,49,50,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("Playstore_final.csv")


In [101]:
list(df2['Version'].value_counts().keys())

['1',
 '1.1',
 'Varies with device',
 '1.2',
 '2',
 '1.0.0',
 '1.3',
 '1.0.1',
 '1.4',
 '1.0.2',
 '3',
 '1.5',
 '1.0.3',
 '1.6',
 '1.0.4',
 '2.1',
 '1.7',
 '4',
 '1.0.5',
 '1.1.0',
 '1.8',
 '5',
 '1.1.1',
 '1.0.6',
 '1.9',
 '2.2',
 '2.0.0',
 '1.0.7',
 '2.3',
 '6',
 '1.0.8',
 '1.1.2',
 '7',
 '1.2.1',
 '1.2.0',
 '3.1',
 '1.1.3',
 '2.4',
 '1.0.9',
 '2.0.1',
 '2.5',
 '3.0.0',
 '1.3.0',
 '1.1.4',
 '1.1.5',
 '3.2',
 '1.2.2',
 '4.1',
 '2.6',
 '1.11',
 '1.3.1',
 '0.1',
 '8',
 '1.4.0',
 '2.1.0',
 '1.01',
 '2.0.2',
 '9.8',
 '2.1.1',
 '2.7',
 '10',
 '2.8',
 '9',
 '1.1.6',
 '1.2.3',
 '1.3.2',
 '1.02',
 '01.01.215',
 '0.0.1',
 '3.3',
 '3.1.0',
 '1.4.1',
 '1.4.25.2',
 '1.12',
 '2.9',
 '4.0.0',
 '3.5',
 '11',
 '2.0.3',
 '1.5.0',
 '1.03',
 '1.1.7',
 '5.1',
 '1.0.10',
 '1.2.4',
 '2.2.0',
 '3.6',
 '1.04',
 '4.2',
 '4.5',
 '1.13',
 '3.0.1',
 '3.4',
 '1.2.5',
 '2.1.2',
 '2.0.4',
 '1.0.11',
 '5.0.0',
 '1.05',
 '1.6.0',
 '1.1.8',
 '1.5.1',
 '1.1.9',
 '2.2.1',
 '0.2',
 '1.15',
 '1.4.2',
 '1.14',
 '1.3.3',
 '

dataset-2

In [136]:

invalids={}
valid_Free= [True, False]
df_2=df2[df2['Free' ].notna()]
invalids["Free"] = df_2[~validate_isin('Free',valid_Free, df_2)]

df_2=df2[df2['Version' ].notna()]
valid_Version = '[A-Za-z.-_\s]*?\s*?_*?-*?.*?v?\d+?(\.\d+)*?' #[A-Za-z]*?\s?-?\s?(\d+(\.\d+)*)?([-_][a-zA-Z0-9]+)*'    (\d+(\.\d+)*)?([-_][a-zA-Z0-9]+)*
invalids['Version' ]= df_2[~validate_pattern('Version',valid_Version, df_2)]

df_2=df2[df2['Developer Internal ID' ].notna()]
df_2['Developer Internal ID'] = df_2['Developer Internal ID'].astype(str)
valid_Developer_Internal_ID = '[0-9]+(?:\.[0-9]+)?(?:[eE][+-]?[0-9]+)?'
invalids['Developer Internal ID' ]= df_2[~validate_pattern('Developer Internal ID',valid_Developer_Internal_ID, df_2)]


df_2=df2[df2['Android version Text' ].notna()] 
valid_Android_version_Text = '\d+(\.\d+)*(\s*-\s*\d+(\.\d+)*)?W*(\d+(\.\d+)*)?\s*(and\s*up)?'#\d+(\.\d+)*\s*-\s*\d+(\.\d+)*|\d+(\.\d+)*\s+and\s+up'
invalids['Android version Text' ]= df_2[~validate_pattern('Android version Text',valid_Android_version_Text, df_2)]

valid_Editor_Choice= [True, False]
df_2=df2[df2['Editor Choice' ].notna()]
invalids["Editor Choice"] = df_2[~validate_isin('Editor Choice',valid_Editor_Choice, df_2)]

valid_In_app_purchases= [True, False]
df_2=df2[df2['In app purchases' ].notna()]
invalids["In app purchases"] = df_2[~validate_isin('In app purchases',valid_In_app_purchases, df_2)]

df_2=df2[df2['Reviews' ].notna()]
invalids['Reviews'] = df_2[~validate_bounded('Reviews',0,np.inf, df_2)]

valid_Ad_Supported= [True]
df_2=df2[df2['Ad Supported' ].notna()]
invalids["Ad Supported"] = df_2[~validate_isin('Ad Supported',valid_Ad_Supported, df_2)]

valid_Content_Rating= ['Everyone','Teen', 'Mature 17+', 'Everyone 10+', 'Adults only 18+', 'Unrated']
df_2=df2[df2['Content Rating' ].notna()]
invalids['Content Rating'] = df_2[~validate_isin('Content Rating',valid_Content_Rating, df_2)]

df_2=df2[df2['Last update' ].notna()]
valid_Last_update	= '\d{1,2}-[(Jan)|(Aug)|Dec|Feb|Jul|Apr|Aug|Mar|May|Nov|Sep|Oct]{3}-\d{2}'
invalids['Last update'] = df_2[~validate_pattern('Last update',valid_Last_update	, df_2)]

df_2=df2[df2['Released' ].notna()]
valid_Released= '\d{1,2}-[(Jan)|(Aug)|Dec|Feb|Jul|Apr|Aug|Mar|May|Nov|Sep|Oct]{3}-\d{2}'
invalids['Released'] = df_2[~validate_pattern('Released',valid_Released, df_2)]

df_2=df2[df2['Developer Email' ].notna()]
valid_Developer_Email= '[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}(?:\.[A-Za-z]{2,})?'
invalids['Developer Email'] = df_2[~validate_pattern('Developer Email',valid_Developer_Email, df_2)]

df_2=df2[df2['Privacy Policy' ].notna()]
valid_Privacy_Policy= '[hH][tT][tT][pP][sS]?://.+'
invalids['Privacy Policy'] = df_2[~validate_pattern('Privacy Policy',valid_Privacy_Policy, df_2)]

df_2=df2[df2['Developer Website' ].notna()]
valid_Developer_Website= '[hH][tT][tT][pP][sS]?://.+'
invalids['Developer Website'] = df_2[~validate_pattern('Developer Website',valid_Developer_Website, df_2)]

df_2=df2[df2['Minimum Android' ].notna()]
valid_Minimum_Android= '\d+(\.\d+)*'
invalids['Minimum Android'] = df_2[~validate_pattern('Minimum Android',valid_Minimum_Android, df_2)]

df_2=df2[df2['Installs' ].notna()]
valid_Installs= '(?:\b\d{1,3}(?:,\d{3})*\+\b)'
invalids['Installs'] = df_2[~validate_pattern('Installs',valid_Installs, df_2)]

df_2=df2[df2['Size' ].notna()]
valid_Size= '\d{1,3}(?:,\d{3})*(?:\.\d+)?[kM]'
invalids['Size'] = df_2[~validate_pattern('Size',valid_Size, df_2)]

df_2=df2[df2['Currency' ].notna()]
valid_Currency= '[A-Z]{3}'
invalids['Currency'] = df_2[~validate_pattern('Currency',valid_Currency, df_2)]

df_2=df2[df2['Price' ].notna()]
df_2['Price'] = df_2['Price'].astype(str)
valid_Price= '(?:\d+(?:\.\d+)?)'
invalids['Price'] = df_2[~validate_pattern('Price',valid_Price, df_2)]

df_2=df2[df2['Minimum Installs' ].notna()]
df_2['Minimum Installs'] = df_2['Minimum Installs'].astype(str)
valid_Minimum_Installs= '(?:\d+(?:\.\d+)?)'
invalids['Minimum Installs'] = df_2[~validate_pattern('Minimum Installs',valid_Minimum_Installs, df_2)]


df_2=df2[df2['Rating' ].notna()]
invalids['Rating'] = df_2[~validate_bounded('Rating',0,5, df_2)]

df_2=df2[df2['Category' ].notna()]
valid_Category = '(?:Education|Tools|Books & Reference|Personalization|Music & Audio|Entertainment|Lifestyle|Business|Productivity|Health & Fitness|Puzzle|Travel & Local|Casual|Photography|Sports|Finance|Simulation|News & Magazines|Arcade|Shopping|Food & Drink|Communication|Action|Educational|Social|Medical|Adventure|Maps & Navigation|Role Playing|Video Players & Editors|Art & Design|Racing|Board|Strategy|Card|Word|Auto & Vehicles|Weather|Trivia|House & Home|Beauty|Dating|Casino|Libraries & Demo|Events|Music|Parenting|Comics)'
invalids['Category'] = df_2[~validate_pattern('Category',valid_Category, df_2)]

df_2=df2[df2['Rating Count' ].notna()]
df_2['Rating Count'] = df_2['Rating Count'].astype(str)
valid_Rating_Count= '\d+\.\d+'
invalids['Rating Count'] = df_2[~validate_pattern('Rating Count',valid_Rating_Count, df_2)]

# invalids['Version'].to_csv('file.csv')

# print(df_2['Rating Count'].dtype)

C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_8368\1302008205.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Developer Internal ID'] = df_2['Developer Internal ID'].astype(str)
C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_8368\1302008205.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Price'] = df_2['Price'].astype(str)
C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_8368\1302008205.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [ ]:
# i=0
# df3 = pd.read_csv("file.csv")
# for i in df3['Version']:
#     if (i!='Varies with device'):
#         print(i)

In [ ]:
# def calculate_consistency(df, column):
#     unique_values_count = df[column].nunique()
#     total_values_count = len(df[column])
    
#     if total_values_count == 0:
#         return 0  # Handle division by zero
#     else:
#         return (unique_values_count / total_values_count) * 100

# def calculate_currentness(df, column):
#     recent_records_count = (df[column] > pd.Timestamp.now() - pd.DateOffset(months=1)).sum()
#     total_records_count = len(df[column])
    
#     if total_records_count == 0:
#         return 0  # Handle division by zero
#     else:
#         return (recent_records_count / total_records_count) * 100

# def calculate_validity(df, column):
#     valid_records_count = df[column].apply(is_valid_email).sum()  # Example function is_valid_email
#     total_records_count = len(df[column])
    
#     if total_records_count == 0:
#         return 0  # Handle division by zero
#     else:
#         return (valid_records_count / total_records_count) * 100

# def calculate_completeness(df, column):
#     non_null_values_count = df[column].count()
#     total_values_count = len(df[column])
    
#     if total_values_count == 0:
#         return 0  # Handle division by zero
#     else:
#         return (non_null_values_count / total_values_count) * 100

# def calculate_accuracy(df, column):
#     accurate_records_count = df[column].apply(is_accurate_value).sum()  # Example function is_accurate_value
#     total_records_count = len(df[column])
    
#     if total_records_count == 0:
#         return 0  # Handle division by zero
#     else:
#         return (accurate_records_count / total_records_count) * 100

# def is_valid_email(email):
#     # Example function to check if an email address is valid
#     return True  # Replace with your implementation

# def is_accurate_value(value):
#     # Example function to check if a value is accurate
#     return True 

# df_tmp2_columns = ['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs', 'Minimum Installs', 'Free', 'Price', 'Currency', 'Size', 'Minimum Android', 'Developer Id', 'Developer Website', 'Developer Email', 'Released', 'Last update', 'Privacy Policy', 'Content Rating', 'Ad Supported', 'In app purchases', 'Editor Choice', 'Summary', 'Reviews', 'Android version Text', 'Developer', 'Developer Address', 'Developer Internal ID', 'Version']
# df_tmp1_columns = ['Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price']

# # Calculate quality for each column in dataset 1 (df1)
# quality_df1 = pd.DataFrame(columns=['Attribute', 'Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy', 'Null Values', 'Records'])

# for column in df_tmp1_columns:
#     consistency = calculate_consistency(df1, column)
#     currentness = calculate_currentness(df1, column)
#     validity = calculate_validity(df1, column)
#     completeness = calculate_completeness(df1, column)
#     accuracy = calculate_accuracy(df1, column)
    
#     null_values = df1[column].isnull().sum()
#     records = len(df1)
    
#     quality_df1 = quality_df1.append({'Attribute': column,'Consistency': consistency,'Currentness': currentness, 'Validity': validity,
#                                       'Completeness': completeness,'Accuracy': accuracy, 'Null Values': null_values, 'Records': records}, ignore_index=True)

# # Calculate quality for each column in dataset 2 (df2)
# quality_df2 = pd.DataFrame(columns=['Attribute', 'Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy', 'Null Values', 'Records'])

# for column in df_tmp2_columns:
#     consistency = calculate_consistency(df2, column)
#     currentness = calculate_currentness(df2, column)
#     validity = calculate_validity(df2, column)
#     completeness = calculate_completeness(df2, column)
#     accuracy = calculate_accuracy(df2, column)
    
#     null_values = df2[column].isnull().sum()
#     records = len(df2)
    
#     quality_df2 = quality_df2.append({'Attribute': column,'Consistency': consistency,'Currentness': currentness,'Validity': validity,
#                                       'Completeness': completeness,'Accuracy': accuracy,'Null Values': null_values, 'Records': records}, ignore_index=True)

# # Calculate percentage of quality for each dataset
# # Quality is calculated as the average of consistency, currentness, validity, completeness, and accuracy
# quality_df1['Quality'] = (quality_df1[['Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy']].sum(axis=1) / 5) * 100
# quality_df2['Quality'] = (quality_df2[['Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy']].sum(axis=1) / 5) * 100

# print("Quality evaluation for Dataset 1:")
# print(quality_df1)

# print("\nQuality evaluation for Dataset 2:")
# print(quality_df2)


In [ ]:
# df_tmp2_columns
# Sample data for df_tmp2
# df_tmp2_columns_data = [
#     ['App1', 'id1', 'Category1', 4.5, 1000, 100000, 50000, True, 0, 'USD', '10M', '4.1', 'dev1', 'www.dev1.com', 'dev1@example.com', '2022-01-01', '2024-03-01', 'http://privacy-policy.com', 'Everyone', True, False, False, 'App1 Summary', 500, 'Android 10', 'Developer1', '123 Street, City, Country', '12345', '1.0.0'],
#     ['App2', 'id2', 'Category2', 4.0, 2000, 500000, 200000, True, 0, 'USD', '15M', '4.4', 'dev2', 'www.dev2.com', 'dev2@example.com', '2022-02-01', '2024-03-15', 'http://privacy-policy.com', 'Everyone', False, True, False, 'App2 Summary', 1000, 'Android 11', 'Developer2', '456 Street, City, Country', '23456', '1.2.0']
# ]

# # Sample data for df_tmp1
# df_tmp1_columns_data = [[4.5, 500, '10M', '100000', 'Free', '0'],[4.0, 1000, '15M', '500000', 'Free', '0']]

# # Creating dataframes
# df_tmp2 = pd.DataFrame(df_tmp2_columns_data, columns=df_tmp2_columns)
# df_tmp1 = pd.DataFrame(df_tmp1_columns_data, columns=df_tmp1_columns)

# # Renaming columns to match df_tmp2
# df_tmp1.columns = ['Rating', 'Rating Count', 'Size', 'Installs', 'Type', 'Price']

# # Merging dataframes
# merged_df = pd.concat([df_tmp2, df_tmp1], ignore_index=True)

# # Output merged dataframe
# print(merged_df)
